In [33]:
%pip install pandas 
%pip install python-dotenv
%pip install -U scikit-learn
%pip install sun_position_calculator
%pip install requests
%pip install docxtpl



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
#local models
from typing import Dict
from models.econometrics import Cost, Currency
from models.components import Component
from models.econometrics import Cost
from models.geometry import Orientation
from models.photovoltaic import Photovoltaic

In [35]:
#local env variables
from dotenv import dotenv_values
config = dotenv_values(".env.local")

# Building references

## Currencies values

In [36]:
import requests # type: ignore
import json
#currency values

# national units https://mindicador.cl/api
query_factor:requests.Response = requests.get('https://mindicador.cl/api',timeout=1000)
ratios_cl = json.loads(query_factor.text)
usd_clp:float = ratios_cl['dolar']['valor']
Cost.set_exchange(Currency.CLP,usd_clp)
Cost.set_exchange(Currency.UF,usd_clp/ratios_cl['uf']['valor']) # 1 dolar in UF
Cost.set_exchange(Currency.UTM,usd_clp/ratios_cl['utm']['valor']) # 1 dolar in Utm

# exchange rates https://app.freecurrencyapi.com/dashboard
query_exchange:requests.Response = requests.get(config["CURRENCY_API_KEY"],timeout=1000)

currency_ratios = json.loads(query_exchange.text)
Cost.set_exchange(Currency.EUR,currency_ratios['data']['EUR'])
Cost.set_exchange(Currency.GBP,currency_ratios['data']['GBP'])
Cost.set_exchange(Currency.BRL,currency_ratios['data']['BRL'])


set $1 Peso Chileno... on USD$0.00 
set $1 UF............. on USD$39.31 
set $1 UTM............ on USD$68.78 
set $1 Euro........... on USD$1.07 
set $1 Pound.......... on USD$1.25 
set $1 Real........... on USD$0.20 


## Address and general info🌎

In [37]:
#cspell:disable


#init proyect
from models.components import Tech
from models.geometry import GeoPosition
from models.inventory import Building, Project

project:Project = Project(
    title='Sistema Fotovoltaico OnGrid-Mixto',
    building = Building(
        geolocation=GeoPosition(latitude=-33.861702,longitude=-71.657839),
        name='Posta de Salud Rural Bucalemu',
        address='Ruta G-80-I km 30 sn, Localidad Bucalemu,\nSanto Domingo',
        city='Santo Domingo'),
    technology= [Tech.PHOTOVOLTAIC]
    )

#project.weather.get_data().sample(10)

api request URL https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M&community=RE&latitude=-33.861702&longitude=-71.657839&start=20230101&end=20231231&format=JSON


# Integration 

## Energy Consumptions🔌

In [38]:
from models.consumption import ElectricityBill, Energetic
project.building.add_consumptions(description="main",energetic=Energetic.ELI,consumption=[
    ElectricityBill(450,"30-12-2022",cost=71400),
    ElectricityBill(497,"30-01-2023",cost=80800),
    ElectricityBill(450,"28-02-2023",cost=71400),
    ElectricityBill(475,"30-03-2023",cost=77400),
    ElectricityBill(475,"30-04-2023",cost=77400),
    ElectricityBill(520,"30-05-2023",cost=93400),
    ElectricityBill(654,"30-06-2023",cost=122100),
    ElectricityBill(820,"30-07-2023",cost=140800),
    ElectricityBill(549,"31-10-2023",cost=100000),
])
project.building.add_consumptions(description="sec",energetic=Energetic.ELI,consumption=[
    ElectricityBill(450,"30-12-2022",cost=71400),
    ElectricityBill(497,"30-01-2023",cost=80800),
    ElectricityBill(450,"28-02-2023",cost=71400),
    ElectricityBill(475,"30-03-2023",cost=77400),
    ElectricityBill(475,"30-04-2023",cost=77400),
    ElectricityBill(520,"30-05-2023",cost=93400),
    ElectricityBill(654,"30-06-2023",cost=122100),
    ElectricityBill(820,"30-07-2023",cost=140800),
    ElectricityBill(549,"31-10-2023",cost=100000),
])

#project.building.consumptions['main'].forecast()
#project.building.consumption_forecast(['main','sec'])

Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True


## Machine Components 🎡

In [39]:
#cspell: disable

#clean previous load
project.components = {}
from IPython.display import clear_output

from models.photovoltaic import Length, PvTechnicalSheet
clear_output(wait=True)

# assembly projects components
project.add_component(
    'generación',
    Photovoltaic(project.weather,'Módulo Primario A',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                quantity=4,
                cost=None,
                orientation=Orientation(15,270-16.46),
                technical_sheet=PvTechnicalSheet(power=655,area=(130.3,238.4,Length.CM)),
                ),
    Photovoltaic(project.weather,'Módulo Secundario B',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                quantity=4,
                cost=None,
                orientation=Orientation(15,16.46),
                technical_sheet=PvTechnicalSheet(power=655,area=(130.3,238.4,Length.CM)),
                )
    )

project.add_component(
    'auxiliar',
    Component('Inversor Híbrido',
                model='Voltronic 5kW 48V',
                reference='https://solarbex.com/comprar/inversor-hibrido-5kw-48v-axpert/',
                specification='Híbrido 5kW Monofásico',
                cost_per_unit=Cost(869,Currency.EUR)),
    Component('Controlador de Carga',
                model='Enertik 80A 12/24V',
                specification='1000W to 2000 24V output',
                cost_per_unit=Cost(101600,Currency.CLP)),
    Component('medidor bidireccional',
                model='Elster AS1440',
                specification='Trifasico',
                cost_per_unit=Cost(306990,Currency.CLP)),
    Component('almacenamiento',
                model='MP NP12-200',
                specification='Batería 12V 200Ah',
                reference='https://www.tiendatecnored.cl/bateria-gel-ciclo-profundo-12v-200ah.html',
                cost_per_unit=Cost(204889,Currency.CLP),
                quantity=5,
                )
    )

project.add_component(
    'obras',
    Component(description='instalación eléctrica',
            specification='H/P eléctrico',
            cost_per_unit=Cost(1,currency=Currency.UF),
            quantity=100,
            ),
    Component(description='estructura de soporte',
            specification='elementos fijación',
            cost_per_unit=Cost((4*800+11400/4+4*700+4000+500)),
            quantity=24
            )
)


Currency.CLP exchange ratios load status: True
inside cost pv :  245990.0 Currency.CLP
Currency.CLP exchange ratios load status: True
inside cost pv :  245990.0 Currency.CLP
Currency.EUR exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.UF exchange ratios load status: True
Currency.CLP exchange ratios load status: True


In [40]:
# psr_bucalemu.components['generación'][1].nominal_power()
#pv_log_model(1500) #clp/w

bucket_list = project.bucket_list(Currency.CLP)

Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.EUR exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.UF exchange ratios load status: True
Currency.CLP exchange ratios load status: True


In [41]:
# psr_bucalemu.get_energy_generation('generación')
project.energy_production('generación')#[['month','day','System_capacity_KW']].groupby(['month','day'],as_index=False).sum()

,Temperature_cell,IRR_incident,System_capacity_KW,date UTC,month,day,hour
0,NaN,NaN,NaN,2022-12-31 19:00:00,12,31,19
1,NaN,NaN,NaN,2022-12-31 20:00:00,12,31,20
2,NaN,NaN,NaN,2022-12-31 21:00:00,12,31,21
3,NaN,NaN,NaN,2022-12-31 22:00:00,12,31,22
4,NaN,NaN,NaN,2022-12-31 23:00:00,12,31,23
...,...,...,...,...,...,...,...
8755,35.305955,885.116396,4.748298,2023-12-31 14:00:00,12,31,14
8756,40.285965,1001.190454,5.234451,2023-12-31 15:00:00,12,31,15
8757,42.527620,1039.564914,5.373605,2023-12-31 16:00:00,12,31,16
8758,38.670189,934.474175,4.938595,2023-12-31 17:00:00,12,31,17


In [42]:
from models.emission import emission

emission.annual_projection(2024)
emission.annual_avg()

project.performance(consumptions=['main'],generation_group='generación')

boundaries in month 8 : 820 <-> 549
boundaries in month 9 : 684.5 <-> 549
boundaries in month 11 : 549 <-> 450


,month,consumption,generation,netbilling,savings,CO2 kg
0,1,497.00,1101.853229,604.853229,497.000000,224.520435
1,2,450.00,893.574547,443.574547,450.000000,182.080281
2,3,475.00,784.602572,309.602572,475.000000,159.875477
3,4,475.00,537.121775,62.121775,475.000000,109.447258
4,5,520.00,378.501318,0.000000,378.501318,77.125772
5,6,654.00,316.784415,0.000000,316.784415,64.549954
6,7,820.00,351.109594,0.000000,351.109594,71.544264
7,8,684.50,422.351444,0.000000,422.351444,86.060945
8,9,616.75,580.386332,0.000000,580.386332,118.263112
9,10,549.00,801.742300,252.742300,549.000000,163.367974


# Write Document 📒


In [43]:
from docxtpl import * 
from datetime import datetime

In [59]:
#assembly proxy object 
ctx:dict[str,any] = {
    #report
    "report_date":datetime.now().strftime("%A, %d de %B %Y"),
    "report_version":"ver.ABR.1",
    #about this project
    "project_name": project.title,
    "project_type" : project.technology[0].value,
    "project_size":project.nominal_power('generación')[0],
    "size_unit":"kW",
    "total_cost": f"{project.bucket_list(Currency.CLP)["cost"]:,}",
    #emissions
    "gas_emission_reduction":f"{project.performance(['main'],'generación')['CO2 kg'].sum():,.2f}",
    "gas_emission_unit":"Kg CO2",
    #site
    "building_name" : project.building.name,
    "city": project.building.city
}
ctx#project.get_context('generación')

Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.EUR exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.UF exchange ratios load status: True
Currency.CLP exchange ratios load status: True
boundaries in month 8 : 820 <-> 549
boundaries in month 9 : 684.5 <-> 549
boundaries in month 11 : 549 <-> 450


{'report_date': 'Monday, 29 de April 2024',
 'report_version': 'ver.ABR.1',
 'project_name': 'Sistema Fotovoltaico OnGrid-Mixto',
 'project_type': 'fotovoltaico',
 'project_size': 5.24,
 'size_unit': 'kW',
 'total_cost': '9,035,925.9',
 'gas_emission_reduction': '1,669.54',
 'gas_emission_unit': 'Kg CO2',
 'building_name': 'Posta de Salud Rural Bucalemu',
 'city': 'Santo Domingo'}

In [52]:
from docxtpl import DocxTemplate
doc = DocxTemplate("templates/memory_template.docx")
doc.render(ctx)
doc.save(f"build/reporte_{project.building.city}_{project.building.name}.docx")

